In [1]:
from IPython.core.display import display, HTML, Markdown
display(HTML("<style>.container { width:98% !important; }</style>"))

In [2]:
%matplotlib inline

In [3]:
import re
import os
import sys
import json
import collections
import geoip2.database
import geoip2.errors
from cachetools import cached
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tldextract
import requests
import netaddr
import datetime
from email_security_providers import *

In [4]:
start_time = datetime.datetime.now()

In [5]:
df = pd.read_csv('mx-intel-enriched.csv').fillna('')

In [6]:
df.head(3).T

,0,1,2
domain,clothes2order.com,famima.vn,brandofsacrifice.com
preference,10,1,5
mailserver,alt4.aspmx.l.google.com,mail.famima.vn,alt2.aspmx.l.google.com
adns_status,ok,ok,ok
adns_code,0,0,0
adns_reason,ok,ok,ok
fail_message,OK,OK,OK
ip_resolutions,['209.85.233.26'],['103.252.255.41'],['142.250.13.26']
maxmind,"[{'ip': '209.85.233.26', 'asn': 15169, 'asname...","[{'ip': '103.252.255.41', 'asn': 45544, 'asnam...","[{'ip': '142.250.13.26', 'asn': 15169, 'asname..."
mailserver_registered_domain,google.com,famima.vn,google.com


In [7]:
display(Markdown('## Top PTR Base Domains'))
pd.DataFrame(df.mx_ip_ptr_registered_domain.value_counts())[:30]

## Top PTR Base Domains

,mx_ip_ptr_registered_domain
,12510098
registrar-servers.com,91762
yandex.ru,68713
mimecast.com,39173
deteque.com,36685
outlook.com,21785
reflexion.net,8959
binero.net,7318
improvmx.com,6988
one.com,5261


In [8]:
display(Markdown('## Top MX Base Domains'))
pd.DataFrame(df.mailserver_registered_domain.value_counts())[:30]

## Top MX Base Domains

,mailserver_registered_domain
google.com,4018222
googlemail.com,1245901
secureserver.net,638784
outlook.com,539918
ovh.net,256850
registrar-servers.com,246956
mailspamprotection.com,197132
one.com,119823
zoho.com,108058
emailsrvr.com,94131


In [9]:
display(Markdown('## Top MX Countries'))
pd.DataFrame(df.maxmind_country.value_counts())[:30]

## Top MX Countries

,maxmind_country
United States,8435220
Germany,719477
France,482903
,478851
United Kingdom,313638
Japan,267244
Russia,220472
Netherlands,181612
Canada,177305
Denmark,152091


In [59]:
display(Markdown('## Top MX Cities'))
df[(df.maxmind_city != '') & (df.maxmind_country != '')][['maxmind_city', 'maxmind_country']].\
    groupby(['maxmind_city', 'maxmind_country']).size().\
    to_frame().reset_index().sort_values(0, ascending=False)[:30]

## Top MX Cities

,maxmind_city,maxmind_country,0
511,Althornbach,Germany,107517
2309,Boardman,United States,89798
21028,Vienna,Austria,66573
2594,Boydton,United States,60932
967,Ashburn,United States,59790
606,Amsterdam,Netherlands,56521
8215,Helsinki,Finland,54116
5375,Dublin,Ireland,53313
13125,Moscow,Russia,52179
5027,Des Moines,United States,51147


In [11]:
display(Markdown('## Top MX ASNs'))
pd.DataFrame(df.maxmind_asname.value_counts())[:30]

## Top MX ASNs

,maxmind_asname
GOOGLE,5249464
AS-26496-GO-DADDY-COM-LLC,657744
MICROSOFT-CORP-MSN-AS-BLOCK,502882
,486686
OVH SAS,358379
NAMECHEAP-NET,335341
1&1 Ionos Se,323412
UNIFIEDLAYER-AS-1,199197
AMAZON-02,191057
GOOGLE-PRIVATE-CLOUD,126473


In [12]:
display(Markdown('## Top MX ASN and MX IP PTRs'))
df[df.mx_ip_ptr_registered_domain != ''][['domain', 'maxmind_asname', 'mx_ip_ptr_registered_domain']].\
    groupby(['maxmind_asname', 'mx_ip_ptr_registered_domain']).\
        count().reset_index().sort_values('domain', ascending=False)[:10]

## Top MX ASN and MX IP PTRs

,maxmind_asname,mx_ip_ptr_registered_domain,domain
6346,NAMECHEAP-NET,registrar-servers.com,91762
11149,YANDEX LLC,yandex.ru,68713
6119,MIMECAST,mimecast.com,39163
2248,DETEQUE,deteque.com,36685
6103,MICROSOFT-CORP-MSN-AS-BLOCK,outlook.com,21785
566,AMAZON-AES,reflexion.net,8959
1286,Binero AB,binero.net,7316
7209,OVH SAS,improvmx.com,6988
7898,One.com A/S,one.com,5261
8349,Paragon Internet Group Limited,webfaction.com,4797


In [13]:
display(Markdown('## Top MX ASN and MX registered domains'))
df[['domain', 'maxmind_asname', 'mailserver_registered_domain']].\
    groupby(['maxmind_asname', 'mailserver_registered_domain']).\
        count().reset_index().sort_values('domain', ascending=False)[:10]

## Top MX ASN and MX registered domains

,maxmind_asname,mailserver_registered_domain,domain
731547,GOOGLE,google.com,3980844
731552,GOOGLE,googlemail.com,1230764
306722,AS-26496-GO-DADDY-COM-LLC,secureserver.net,627438
1125750,MICROSOFT-CORP-MSN-AS-BLOCK,outlook.com,492562
1292095,OVH SAS,ovh.net,256228
1159353,NAMECHEAP-NET,registrar-servers.com,246046
743099,GOOGLE-PRIVATE-CLOUD,mailspamprotection.com,126416
1316551,One.com A/S,one.com,118895
1944278,ZOHO-AS,zoho.com,107503
1399908,RACKSPACE,emailsrvr.com,93884


In [14]:
display(Markdown('## Top MX registered domains and MX IP PTRs'))
domain_ptr = df[['domain', 'mailserver_registered_domain', 'mx_ip_ptr_registered_domain']].\
    groupby(['mailserver_registered_domain', 'mx_ip_ptr_registered_domain']).\
        count().reset_index()
domain_ptr[
    (domain_ptr.mailserver_registered_domain != domain_ptr.mx_ip_ptr_registered_domain) & 
    (domain_ptr.mx_ip_ptr_registered_domain != '')
].sort_values('domain', ascending=False)

## Top MX registered domains and MX IP PTRs

,mailserver_registered_domain,mx_ip_ptr_registered_domain,domain
1917606,yandex.net,yandex.ru,52431
1866943,web-hosting.com,registrar-servers.com,45250
780988,hostedmxserver.com,deteque.com,16695
1090567,mb1p.com,deteque.com,9264
1047379,m2bp.com,deteque.com,9056
...,...,...,...
684031,gimmond.co.jp,sakura.ne.jp,1
684067,gimnazijamostar.ba,global.ba,1
684071,gimnazium47.ru,ural.net,1
684118,gims-ph.com,secureserver.net,1


In [15]:
domain_ptr[domain_ptr.mx_ip_ptr_registered_domain == 'amazonaws.com'].sort_values('domain', ascending=False)

,mailserver_registered_domain,mx_ip_ptr_registered_domain,domain
1388661,protectedservice.net,amazonaws.com,371
1056427,mailchannels.net,amazonaws.com,184
1600022,spamstore.co.uk,amazonaws.com,33
1056352,mailbagger.com,amazonaws.com,32
87948,amazonaws.com,amazonaws.com,21
...,...,...,...
699867,goo-b.jp,amazonaws.com,1
685083,giornataparkinson.it,amazonaws.com,1
656528,gadgetm.jp,amazonaws.com,1
653765,futuro-rio.com,amazonaws.com,1


In [16]:
domain_ptr[domain_ptr.mx_ip_ptr_registered_domain == 'amazonaws.com'].domain.sum()

1052

In [17]:
domain_ptr[domain_ptr.mx_ip_ptr_registered_domain == 'googleusercontent.com'].sort_values('domain', ascending=False)

,mailserver_registered_domain,mx_ip_ptr_registered_domain,domain
629512,fortifi.zone,googleusercontent.com,6
1604259,spillgroup.com,googleusercontent.com,4
1582744,sobefest.com,googleusercontent.com,2
1287298,osteriadelpescefirenze.it,googleusercontent.com,2
1262748,okose.com,googleusercontent.com,1
...,...,...,...
658721,galleon-wallet.tech,googleusercontent.com,1
652895,futbol10shop.com,googleusercontent.com,1
634697,francescadebardin.com,googleusercontent.com,1
633295,foxtalebookshoppe.com,googleusercontent.com,1


In [18]:
domain_ptr[domain_ptr.mx_ip_ptr_registered_domain == 'googleusercontent.com'].domain.sum()

400

In [19]:
df[df.mx_ip_soa_nameserver != ''][['domain', 'maxmind_asname', 'mx_ip_soa_nameserver']].\
    groupby(['maxmind_asname', 'mx_ip_soa_nameserver']).\
        count().reset_index().sort_values('domain', ascending=False)[:10]

,maxmind_asname,mx_ip_soa_nameserver,domain
44,PROOFPOINT-ASN-US-WEST,ns1.proofpoint.com,3203
43,PROOFPOINT-ASN-US-EAST,ns1.proofpoint.com,2151
45,"Proofpoint, Inc.",ns1.proofpoint.com,1734
54,UNIFIEDLAYER-AS-1,ns1.unifiedlayer.com,1520
30,IRONPORT-SYSTEMS-INC,ns1-93.akam.net,1494
5,ASMALLORANGE1,dns1.athenixinc.com,515
7,AUTOMATTIC,ns1.automattic.com,360
36,Ltd Hostpro Lab,primary.hostsila.com,190
32,LEVEL3,ns1.iservice.net,173
25,HVC-AS,rdns1.hivelocity.net,114


In [20]:
pd.DataFrame(df.mx_ip_soa_nameserver.value_counts())[:10]

,mx_ip_soa_nameserver
,12897965
ns1.proofpoint.com,7088
ns1.unifiedlayer.com,1521
ns1-93.akam.net,1494
dns1.athenixinc.com,517
ns1.automattic.com,360
primary.hostsila.com,190
ns1.iservice.net,173
rdns1.hivelocity.net,114
ns1.servercentral.net,111


In [21]:
pd.DataFrame(df.mx_ip_soa_hostmaster.value_counts())[:10]

,mx_ip_soa_hostmaster
,12897965
ops@proofpoint.com,5715
abuse@unifiedlayer.com,1521
"""stbu-hostmaster@cisco""@com",1494
x-ops@proofpoint.com,1373
servers@asonoc.com,517
hostmaster@automattic.com,360
admin@hostpro.ua,190
root@ns.inmotionhosting.com,175
hostmaster@iservice.net,173


In [22]:
display(Markdown('### MX Domains whose IPs are hosted in Azure'))
pd.DataFrame(df[df.is_azure].groupby(['mailserver_registered_domain']).size()).reset_index().sort_values(0, ascending=False)[:20]

### MX Domains whose IPs are hosted in Azure

,mailserver_registered_domain,0
743,icoremail.net,364
1350,scanscope.net,310
955,mailinblack.com,259
236,cali.co.uk,141
209,brightberri.net,141
454,dotmailer.co.uk,130
997,menufy.com,120
467,dsmail.es,91
694,helionmail.com,89
147,azure.com,44


In [23]:
display(Markdown('### MX Domains whose IPs are hosted in AWS'))
pd.DataFrame(df[df.is_aws].groupby(['mailserver_registered_domain']).size()).reset_index().sort_values(0, ascending=False)[:20]

### MX Domains whose IPs are hosted in AWS

,mailserver_registered_domain,0
8300,h-email.net,27200
11971,mailgun.org,25229
15314,pickelhost.com,17665
12569,messagelabs.com,13035
960,amazonaws.com,11657
1902,b-io.co,6448
2051,barracudanetworks.com,4394
18116,sophos.com,4197
20226,trendmicro.com,4136
11970,mailguard.com.au,4106


In [24]:
display(Markdown('### MX Domains whose IPs are hosted in GCP'))
pd.DataFrame(df[df.is_gcp].groupby(['mailserver_registered_domain']).size()).reset_index().sort_values(0, ascending=False)[:20]

### MX Domains whose IPs are hosted in GCP

,mailserver_registered_domain,0
6239,mailspamprotection.com,62642
7132,neen.it,171
5261,jouwweb.nl,143
1828,ccnotifier.nl,59
9228,siteground.biz,44
4004,getontheweb.com,40
10923,uservers.net,37
9053,sgvps.net,33
1696,capnova.com,24
6234,mailcannon.net,18


In [25]:
def domain_count_by_key(p_df, keyname='email_provider'):
    '''
    Conceptually similar to this SQL query:
        SELECT 
            keyname, 
            COUNT(DISTINCT(domain)) as count
        FROM 
            p_df
        WHERE
            keyname!=''
        GROUP BY
            keyname
        ORDER BY
            count DESC
    '''
    return p_df[p_df[keyname] != ''][[keyname, 'domain']].\
            drop_duplicates().\
            groupby([keyname]).\
            size().\
            sort_values(0, ascending=False).\
            to_frame().\
            reset_index().\
            rename(columns={0: 'count'})

In [26]:
display(Markdown('### Top Email Security Providers'))
results = domain_count_by_key(df)
results.to_csv('results/Top Email Security Providers.csv', index=False)
results

### Top Email Security Providers

,email_provider,count
0,Proofpoint,42310
1,Mimecast,36064
2,Deteque,34759
3,Barracuda,22897
4,Solarwinds,14681
5,Symantec,13832
6,AppRiver,11149
7,Protonmail,8158
8,Trend Micro,8055
9,Cisco Ironport,7504


In [27]:
display(Markdown('### Top Email Providers (as determined by ASN Name)'))
results = domain_count_by_key(df, 'email_provider_from_asnname')
results.to_csv('results/Top Email Providers (as determined by ASN Name).csv', index=False)
results

### Top Email Providers (as determined by ASN Name)

,email_provider_from_asnname,count
0,Proofpoint,40571
1,Mimecast,35385
2,Deteque,34759
3,AppRiver,10081
4,Symantec,7320
5,Cisco Ironport,6982
6,Symantec,6887
7,Protonmail,4557
8,Forcepoint,3680
9,hornetsecurity,2009


In [28]:
display(Markdown('### Top Email Security Providers (as determined by MX Domain)'))
results = domain_count_by_key(df, 'email_provider_from_mailserver')
results.to_csv('results/Top Email Security Providers (as determined by MX Domain).csv', index=False)
results

### Top Email Security Providers (as determined by MX Domain)

,email_provider_from_mailserver,count
0,Proofpoint,41735
1,Mimecast,36055
2,Barracuda,22885
3,Symantec,13813
4,Solarwinds,11973
5,AppRiver,11143
6,Protonmail,8156
7,Cisco Ironport,7124
8,Trend Micro,7047
9,CSC Digital Brand Services,5074


In [29]:
display(Markdown('### Top Email Security Providers (as determined by PTR Domain)'))
results = domain_count_by_key(df, 'email_provider_from_ptr')
results.to_csv('results/Top Email Security Providers (as determined by PTR Domain).csv', index=False)
results

### Top Email Security Providers (as determined by PTR Domain)

,email_provider_from_ptr,count
0,Deteque,27050
1,Mimecast,20100
2,Symantec,4206
3,Reflexion,3631
4,Trend Micro,3582
5,Solarwinds,2709
6,Proofpoint,1146
7,the email laundry,447
8,MXGuarddog,408
9,Panda Security,334


In [30]:
display(Markdown('### Top Email Security Providers Hosted in AWS'))
results = domain_count_by_key(df[df.is_aws])
results.to_csv('results/Top Email Security Providers Hosted in AWS.csv', index=False)
results

### Top Email Security Providers Hosted in AWS

,email_provider,count
0,Symantec,12864
1,Trend Micro,6494
2,Barracuda,2253
3,Sophos,2114
4,vadesecure,1583
5,Mailprotector,1234
6,FireEye,963
7,DuoCircle,921
8,SpamTitan,662
9,AVG,393


In [31]:
display(Markdown('### Top Email Providers Hosted in Azure'))
results = domain_count_by_key(df[df.is_azure])
results.to_csv('results/Top Email Security Providers Hosted in Azure.csv', index=False)
results

### Top Email Providers Hosted in Azure

,email_provider,count
0,Mail in Black,244
1,Censornet,200


In [32]:
display(Markdown('### Top Email Providers Hosted in GCP'))
domain_count_by_key(df[df.is_gcp | df.maxmind_asname.str.contains('GOOGLE')])

### Top Email Providers Hosted in GCP

,email_provider,count


In [33]:
display(Markdown('### Top Email Security Providers (self-hosted)'))
results = domain_count_by_key(df[df.email_provider_from_asnname != ''])
results.to_csv('results/Top Email Security Providers (self-hosted).csv', index=False)
results

### Top Email Security Providers (self-hosted)

,email_provider,count
0,Proofpoint,40571
1,Mimecast,35385
2,Deteque,34759
3,Symantec,13523
4,AppRiver,10081
5,Cisco Ironport,6982
6,Protonmail,4557
7,Forcepoint,3680
8,hornetsecurity,2009
9,Trend Micro,1411


In [34]:
display(Markdown('### Top Email Security Providers hosted in Linode'))
domain_count_by_key(df[df.maxmind_asname == 'Linode, LLC'])

### Top Email Security Providers hosted in Linode

,email_provider,count
0,CSC Digital Brand Services,5043


In [35]:
display(Markdown('### Top Email Security Providers hosted in Digital Ocean'))
domain_count_by_key(df[df.maxmind_asname == 'DIGITALOCEAN-ASN'])

### Top Email Security Providers hosted in Digital Ocean

,email_provider,count


In [36]:
display(Markdown('### Top Email Security Providers hosted in Rackspace'))
domain_count_by_key(df[df.maxmind_asname == 'RACKSPACE'])

### Top Email Security Providers hosted in Rackspace

,email_provider,count


In [37]:
display(Markdown('### Top Non-self hosted ASNs of Email Security Providers'))
results = df[(df.email_provider != '') & (~df.maxmind_asname.isin(email_provider_asns.keys())) & (df.maxmind_asname != '')].\
    maxmind_asname.value_counts().to_frame().reset_index()[:1000]
results.to_csv('results/Top Non-self hosted ASNs of Email Security Providers.csv', index=False)
results[:30]

### Top Non-self hosted ASNs of Email Security Providers

,index,maxmind_asname
0,AMAZON-02,68406
1,AMAZON-AES,25547
2,COGENT-174,15116
3,Hostway Deutschland GmbH,6347
4,"Linode, LLC",5043
5,ZC38-AS1,4855
6,DATABANK-SLC,4854
7,j2 Global Ireland Limited,3447
8,CENTURYLINK-LEGACY-SAVVIS,3446
9,MK Netzdienste GmbH & Co. KG,3187


In [38]:
display(Markdown('### Top unlabeled MX registered domains'))
results = df[(~df.mailserver_registered_domain.isin(email_provider_domains.keys()))].\
    mailserver_registered_domain.value_counts().to_frame().reset_index()[:1000]
results.to_csv('results/Top unlabeled MX registered domains.csv', index=False)
results[:30]

### Top unlabeled MX registered domains

,index,mailserver_registered_domain
0,google.com,4018222
1,googlemail.com,1245901
2,secureserver.net,638784
3,outlook.com,539918
4,ovh.net,256850
5,registrar-servers.com,246956
6,mailspamprotection.com,197132
7,one.com,119823
8,zoho.com,108058
9,emailsrvr.com,94131


In [39]:
f1000 = [domain.strip() for domain in open('data/f1000-domains.txt')]
f100 = f1000[:100]
f50 = f1000[:100]
f10 = f1000[:10]

In [40]:
pd.options.display.max_rows = 100
display(Markdown('### Fortune 1000 Email Security Providers'))
results = domain_count_by_key(df[df.domain.isin(f1000)])
results.to_csv('results/Fortune 1000 Email Security Providers.csv', index=False)
results

### Fortune 1000 Email Security Providers

,email_provider,count
0,Proofpoint,340
1,Cisco Ironport,75
2,Mimecast,65
3,Symantec,54
4,FireEye,14
5,Trend Micro,4
6,Forcepoint,4
7,CSC Digital Brand Services,4
8,Barracuda,4
9,Postini,2


In [41]:
display(Markdown('### Fortune 100 Email Security Providers'))
results = domain_count_by_key(df[df.domain.isin(f100)])
results.to_csv('results/Fortune 100 Email Security Providers.csv', index=False)
results

### Fortune 100 Email Security Providers

,email_provider,count
0,Proofpoint,35
1,Cisco Ironport,8
2,Symantec,5
3,CSC Digital Brand Services,2
4,Mimecast,1
5,FireEye,1


In [42]:
display(Markdown('### Fortune 50 Email Security Providers'))
results = domain_count_by_key(df[df.domain.isin(f50)])
results.to_csv('results/Fortune 50 Email Security Providers.csv', index=False)
results

### Fortune 50 Email Security Providers

,email_provider,count
0,Proofpoint,35
1,Cisco Ironport,8
2,Symantec,5
3,CSC Digital Brand Services,2
4,Mimecast,1
5,FireEye,1


In [43]:
display(Markdown('### Fortune 10 Email Security Providers'))
results = domain_count_by_key(df[df.domain.isin(f10)])
results.to_csv('results/Fortune 10 Email Security Providers.csv', index=False)
results

### Fortune 10 Email Security Providers

,email_provider,count
0,Proofpoint,3
1,Symantec,1


In [44]:
display(Markdown('### Fortune 10 Summary'))
df[df.domain.isin(f10)][['domain', 'mailserver_registered_domain', 'email_provider']].drop_duplicates().sort_values('email_provider')

### Fortune 10 Summary

,domain,mailserver_registered_domain,email_provider
895915,gm.com,gm.com,
1003703,apple.com,apple.com,
1019948,berkshirehathaway.com,outlook.com,
3214516,exxonmobil.com,exxonmobil.com,
5247656,chevron.com,chevron.com,
283064,walmart.com,pphosted.com,Proofpoint
364491,ge.com,pphosted.com,Proofpoint
2980609,cvshealth.com,pphosted.com,Proofpoint
6274,ford.com,messagelabs.com,Symantec


In [45]:
display(Markdown('### Top Mailserver 2LDs'))
results = domain_count_by_key(df, 'mailserver_registered_domain')[:1000]
results.to_csv('results/Top Mailserver 2LDs.csv', index=False)
results[:20]

### Top Mailserver 2LDs

,mailserver_registered_domain,count
0,google.com,1031219
1,outlook.com,530855
2,googlemail.com,511455
3,secureserver.net,320525
4,ovh.net,102265
5,mailspamprotection.com,65745
6,yandex.net,51512
7,emailsrvr.com,47438
8,registrar-servers.com,46698
9,dreamhost.com,45791


In [46]:
display(Markdown('### Top Mailserver 2LDs hosted in AWS'))
results = domain_count_by_key(df[df.is_aws], 'mailserver_registered_domain')[:1000]
results.to_csv('results/Top Mailserver 2LDs hosted in AWS.csv', index=False)
results[:20]

### Top Mailserver 2LDs hosted in AWS

,mailserver_registered_domain,count
0,h-email.net,27184
1,pickelhost.com,17653
2,messagelabs.com,12864
3,mailgun.org,12763
4,amazonaws.com,11430
5,b-io.co,6437
6,trendmicro.com,3970
7,trendmicro.eu,2525
8,mailerhost.net,2522
9,barracudanetworks.com,2204


In [47]:
display(Markdown('### Top Mailserver 2LDs hosted in Azure'))
results = domain_count_by_key(df[df.is_azure], 'mailserver_registered_domain')[:1000]
results.to_csv('results/Top Mailserver 2LDs hosted in Azure.csv', index=False)
results[:20]

### Top Mailserver 2LDs hosted in Azure

,mailserver_registered_domain,count
0,icoremail.net,355
1,mailinblack.com,244
2,scanscope.net,200
3,cali.co.uk,141
4,menufy.com,120
5,dotmailer.co.uk,66
6,helionmail.com,64
7,azure.com,42
8,brightberri.net,37
9,dsmail.es,32


In [48]:
display(Markdown('### Top Mailserver 2LDs hosted in GCP'))
results = domain_count_by_key(df[df.is_gcp], 'mailserver_registered_domain')[:1000]
results.to_csv('results/Top Mailserver 2LDs hosted in GCP.csv', index=False)
results[:20]

### Top Mailserver 2LDs hosted in GCP

,mailserver_registered_domain,count
0,mailspamprotection.com,44678
1,jouwweb.nl,143
2,neen.it,80
3,ccnotifier.nl,59
4,getontheweb.com,40
5,uservers.net,37
6,siteground.biz,32
7,sgvps.net,30
8,capnova.com,24
9,mailcannon.net,18


In [49]:
for domainname, provider in email_provider_domains.items():
    tmp = df[df.mailserver_registered_domain == domainname]
    tmp = tmp[tmp.mailserver_registered_domain_nameserver1 != '']
    if len(tmp.mailserver_registered_domain_nameserver1.unique()) > 0:
        print('{} Nameservers for {} ({})'.format(len(tmp.mailserver_registered_domain_nameserver1.unique()), domainname, provider))
        print(tmp.mailserver_registered_domain_nameserver1.value_counts())
        print('---\n')

1 Nameservers for activegate-ss.jp (Activegate SS)
ns-1269.awsdns-30.org    73
Name: mailserver_registered_domain_nameserver1, dtype: int64
---

1 Nameservers for antispameurope.com (hornetsecurity)
godzilla-haj2.antispameurope.de    8201
Name: mailserver_registered_domain_nameserver1, dtype: int64
---

1 Nameservers for appriver.com (AppRiver)
hugh.ns.cloudflare.com    331
Name: mailserver_registered_domain_nameserver1, dtype: int64
---

1 Nameservers for arsmtp.com (AppRiver)
mdns1.appriver.com    21902
Name: mailserver_registered_domain_nameserver1, dtype: int64
---

1 Nameservers for avgcloud.net (AVG)
ns-1350.awsdns-40.org    789
Name: mailserver_registered_domain_nameserver1, dtype: int64
---

1 Nameservers for baesystems.com (BAE Systems)
udns1.cscdns.net    11
Name: mailserver_registered_domain_nameserver1, dtype: int64
---

1 Nameservers for barracudanetworks.com (Barracuda)
dina.ns.cloudflare.com    45495
Name: mailserver_registered_domain_nameserver1, dtype: int64
---

1 Nam

1 Nameservers for vadesecure.com (vadesecure)
ns-1471.awsdns-55.org    6418
Name: mailserver_registered_domain_nameserver1, dtype: int64
---

1 Nameservers for watchguard.com (Watchguard)
ns1.dnsimple.com    1
Name: mailserver_registered_domain_nameserver1, dtype: int64
---

1 Nameservers for zixmail.net (ZixMail)
ns01.zixcorp.com    19
Name: mailserver_registered_domain_nameserver1, dtype: int64
---

1 Nameservers for zixsmbhosted.com (ZixMail)
ns01.zixcorp.com    23
Name: mailserver_registered_domain_nameserver1, dtype: int64
---

1 Nameservers for psmtp.com (Postini)
ns1.google.com    6241
Name: mailserver_registered_domain_nameserver1, dtype: int64
---

1 Nameservers for mailinblack.com (Mail in Black)
ns-110-a.gandi.net    3558
Name: mailserver_registered_domain_nameserver1, dtype: int64
---

1 Nameservers for emailservice.io (Mailprotector)
ns-1396.awsdns-46.org    1336
Name: mailserver_registered_domain_nameserver1, dtype: int64
---

1 Nameservers for cscdns.net (CSC Digital Bra

In [50]:
def provider_search(search):
    '''
    provider a mailserver domain name or string, and get a summary of the data we have about it
    '''
    tmp = df[['domain', 'mailserver', 'mailserver_registered_domain', 'maxmind_asname']][df.mailserver.str.contains(search) | df.maxmind_asname.str.contains(search)]
    print(tmp.mailserver_registered_domain.value_counts())
    print('---')
    print(tmp.mailserver.value_counts())
    print('---')
    print(tmp.maxmind_asname.value_counts())
    print('---')
    print(tmp.domain.value_counts())

In [51]:
def explore_mailservers_by_known_asns():
    for asn, provider in email_provider_asns.items():
        tmp = df[df.maxmind_asname == asn]
        print('MXs for {} ({})'.format(asn, provider))
        print(tmp.mailserver_registered_domain.value_counts())
        print('---')

In [52]:
def explore_asns_by_known_mailservers():
    for domainname, provider in email_provider_domains.items():
        tmp = df[df.mailserver_registered_domain == domainname]
        print('ASNs for {} ({})'.format(domainname, provider))
        print(tmp.maxmind_asname.value_counts())
        print('---')

In [53]:
def identify_more_provider_asns():
    import textdistance
    asns = df.maxmind_asname.map(lambda asn: str(asn)).unique()
    for asname,provider in email_provider_asns.items():
        count = 0
        for asn in asns:
            asname_score = textdistance.jaro_winkler(asn.lower(), asname.lower())
            provider_score = textdistance.jaro_winkler(asn.lower(), provider.lower())
            if asname.lower() in asn.lower() or provider.lower() in asn.lower() or asname_score > 0.8 or provider_score > 0.8:
                count += 1
                print('"{}": "{}",'.format(asn, provider))
        if count == 0:
            print('NONE FOUND: "{}" ("{}")'.format(asname,provider))

In [54]:
display(Markdown('### Top Mailserver Nameservers'))
results = df[df.mailserver_registered_domain_nameserver1 != '']['mailserver_registered_domain_nameserver1'].\
    value_counts().to_frame()[:1000]
results.to_csv('results/Top Mailserver Nameservers.csv', index=False)
results[:30]

### Top Mailserver Nameservers

,mailserver_registered_domain_nameserver1
ns1.google.com,5270713
a1-245.akam.net,638790
ns1.msft.net,540042
ns-1and1.ui-dns.biz,295656
edns4.ultradns.biz,264138
dns10.ovh.net,256981
ns1.clev1.net,197145
dns1.p03.nsone.net,132351
a.b-one-dns.net,119823
ns1.p256.dynect.net,94131


In [55]:
display(Markdown('### Quick summary reports for analysis/exploration ...'))
#mask = df.mailserver_registered_domain_nameserver1.fillna('').str.contains('cscdns.net')
#mask = df.maxmind_asname == 'Linode, LLC'
#mask = df.maxmind_asname == 'DIGITALOCEAN-ASN'
#mask = df.maxmind_asname == 'RACKSPACE'
#mask = df.maxmind_asname.fillna('').str.contains('Alibaba')
#mask = df.mailserver_registered_domain == 'h-email.net'
#mask = df.mx_ip_ptr_registered_domain == 'deteque.com'
#mask = df.mx_ip_ptr_registered_domain == 'spamcloud.md'
#mask = df.domain.isin(f100)
mask = df.mailserver_registered_domain == 'cscdns.net'

display(df[mask].mailserver_registered_domain.value_counts().to_frame())
display(df[mask].maxmind_asname.value_counts().to_frame())
display(df[mask].mx_ip_ptr_registered_domain.value_counts().to_frame())
display(df[mask].mailserver_registered_domain_nameserver1.value_counts().to_frame())
display(df[mask].email_provider.value_counts().to_frame())
display(df[mask].mailserver.value_counts().to_frame())
#display(df[mask].domain.value_counts().to_frame())

### Quick summary reports for analysis/exploration ...

,mailserver_registered_domain
cscdns.net,5077


,maxmind_asname
"Linode, LLC",5043
,34


,mx_ip_ptr_registered_domain
,5077


,mailserver_registered_domain_nameserver1
udns1.cscdns.net,5077


,email_provider
CSC Digital Brand Services,5077


,mailserver
custmx.cscdns.net,5077


In [56]:
end_time = datetime.datetime.now()
print(start_time)
print(end_time)

2020-06-27 14:34:01.122015
2020-06-27 14:41:16.592097


In [ ]:
def fix(row):
    if row['mailserver_registered_domain'] == 'outlook.com':
        row['email_provider'] = 'Microsoft'
    if row['mailserver_registered_domain'] in ('google.com', 'googlemail.com'):
        row['email_provider'] = 'Google'
    return row

In [70]:
results = df[df.domain.isin(f100)][['domain', 'mailserver_registered_domain', 'email_provider']].\
    drop_duplicates().replace('', '-')
results = pd.DataFrame(list([fix(row.to_dict()) for idx, row in results.iterrows()])).sort_values('email_provider')
results.to_csv('results/Fortune 100 Listing.csv', index=False)
display(Markdown('### Fortune 100 Listing Domain, MX Base Domain, Security Provider'))
results

### Fortune 100 Listing Domain, MX Base Domain, Security Provider

,domain,mailserver_registered_domain,email_provider
91,coca-colacompany.com,ko.com,-
75,searsholdings.com,searsholdings.com,-
25,apple.com,apple.com,-
79,amazon.com,amazon.com,-
23,unitedhealthgroup.com,corpmailsvcs.com,-
22,gm.com,gm.com,-
36,lockheedmartin.com,lmco.com,-
73,prudential.com,prudential.com,-
70,pepsico.com,pepsico.com,-
82,target.com,target.com,-


In [71]:
results = df[df.alexa_domain.map(lambda rank: int(rank) if rank else 10000000) <= 100][['domain', 'mailserver_registered_domain', 'email_provider']].\
    drop_duplicates().replace('', '-')
results = pd.DataFrame(list([fix(row.to_dict()) for idx, row in results.iterrows()])).sort_values('email_provider')
results.to_csv('results/Alexa 100 Listing.csv', index=False)
display(Markdown('### Alexa 100 Listing Domain, MX Base Domain, Security Provider'))
results.head(100)

### Alexa 100 Listing Domain, MX Base Domain, Security Provider

,domain,mailserver_registered_domain,email_provider
106,aliexpress.com,aliyun.com,-
58,wordpress.com,automattic.com,-
83,amazon.de,amazon.com,-
82,wikipedia.org,wikimedia.org,-
79,jd.com,jd.com,-
77,taobao.com,alibaba-inc.com,-
76,tribunnews.com,tribunnews.com,-
75,paypal.com,paypalcorp.com,-
37,naver.com,naver.com,-
39,iqiyi.com,iqiyi.com,-


In [74]:
display(Markdown('### Alexa 1000 Email Security Providers'))
results = domain_count_by_key(df[df.alexa_domain.map(lambda rank: int(rank) if rank else 10000000) <= 100])[:1000]
results.to_csv('results/Alexa 1000 Email Security Providers.csv', index=False)
results[:20]

### Alexa 1000 Email Security Providers

,email_provider,count
0,Proofpoint,4
1,Symantec,1
2,Microsoft Frontbridge,1
3,Cisco Ironport,1


In [83]:
print(f'''
* {df.shape[0]:0,.0f} unique MX records (from {len(df.domain.unique())} unique domains)
* {len(df.mailserver.unique()):0,.0f} Unique Mailserver domains
* {len(df.mailserver_registered_domain.unique()):0,.0f} Unique Mailserver base domains
* {len(df.maxmind_asn.unique()):0,.0f} Unique Mailserver ASNs
* {len(df.email_provider.unique()):0,.0f} Unique Security Providers identified
'''
)


* 12,910,550 unique MX records (from 5994452 unique domains)
* 2,901,843 Unique Mailserver domains
* 1,940,993 Unique Mailserver base domains
* 25,733 Unique Mailserver ASNs
* 56 Unique Security Providers identified

